In [7]:
import os
import json
import glob
import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import skimage

In [ ]:
from ..problems.modir3d.sitk_utils import *

In [6]:
root_dir = "/export/scratch2/data/grewal/Data/Projects_JPG_data/MO_DIR/LUMC_cervical_test_annotated/MODIR16"

filepaths = glob.glob(root_dir + "*/*.json")
print(len(filepaths))
for filepath in filepaths:
    print(filepath)
    info = json.load(open(filepath, "r"))

    labels = [item["label_name"] for item in info["annotation"]]
    print(np.unique(labels))
    print("")

2
/export/scratch2/data/grewal/Data/Projects_JPG_data/MO_DIR/LUMC_cervical_test_annotated/MODIR16/2.25.288064712475147460804240590614900732253.json
['a_uterina_left' 'a_uterina_right' 'ac_left' 'ac_right' 'asbs' 'bladder'
 'body' 'bowel' 'bowel (1) mri' 'cervix_os' 'coccygis' 'ext_urethral_os'
 'fh_left' 'fh_right' 'gtv_res' 'hr_ctv' 'int_anal_sfinct'
 'int_urethral_os' 'ir_ctv' 'isthmus' 'iu_canal_top' 'pibs_os' 'rectum'
 'rotundum_left' 'rotundum_right' 's2s3' 's3s4' 'sigmoid'
 'ureteral_os_left' 'ureteral_os_right' 'uterus-vagina' 'uterus_top']

/export/scratch2/data/grewal/Data/Projects_JPG_data/MO_DIR/LUMC_cervical_test_annotated/MODIR16/2.25.89225162996425888780919817002874880455.json
['a_uterina_left' 'a_uterina_right' 'ac_left' 'ac_right' 'asbs' 'bladder'
 'bowel' 'cervix_os' 'coccygis' 'ext_urethral_os' 'fh_left' 'fh_right'
 'gtv_res' 'hr_ctv' 'int_anal_sfinct' 'int_urethral_os' 'ir_ctv' 'isthmus'
 'iu_canal_top' 'pibs_os' 'rectum' 'rotundum_left' 'rotundum_right' 's2s3'
 's3s

In [ ]:
def visualize_label(meta_list, annotations, label_pp, view="transverse"):
	# colors = {0: (1, 0, 0), 1: (1, 0, 1), 2: (0, 1, 0), 3: (0, 0, 1),
	# 			4: (1, 1, 0), 5: (0, 1, 1), 6: (1, 0, 1),
	# 			7: (1, 0.5, 0), 8: (0, 1, 0.5), 9: (0.5, 0, 1),
	# 			10: (0.5, 1, 0), 11: (0, 0.5, 1), 12: (1, 0, 0.5)}
	meta_sorted = sorted(meta_list, key=lambda x: x['SliceLocation'])
	for i, meta in enumerate(meta_sorted):
		dicom_path = meta['original_path']
		im = pydicom.dcmread(dicom_path)
		img = process_dicom_array(im, meta)

		img = np.stack((img,)*3, axis=-1)
		combined = img.copy()
		opacity = 0.8
		uid = meta['uid']
		annotation = [item for item in annotations if item["uid"]==uid]
		for idx, item in enumerate(annotation):
			coords = item["coords"]
			coords = np.asarray(coords).squeeze()
			rr, cc = skimage.draw.polygon(coords[:,0], coords[:,1], shape=img.shape)
			combined[cc, rr] = opacity*np.array(item["color"]) + (1-opacity)*combined[cc, rr]
		
		combined = np.concatenate((combined, img), axis=1)
		output_path = str(label_pp) + f"/{i}.jpg"
		imsave(output_path, (combined*255).astype(np.uint8))

In [ ]:
info = json.load(open(filepaths[0], "r"))
fixed_meta = info["meta"]
fixed_annotations = info["annotation"]
fixed_meta = sorted(fixed_meta, key=lambda x: x["SliceLocation"])
fixed_impaths = [item["original_path"] for item in fixed_meta]

info = json.load(open(filepath[1], "r"))
moving_meta = info["meta"]
moving_annotations = info["annotation"]
moving_meta = sorted(moving_meta, key=lambda x: x["SliceLocation"])
moving_impaths = [item["original_path"] for item in moving_meta]

fixed_image = read_image(fixed_impaths, output_spacing=None, crop_depth=False, rescaling=True)
moving_image = read_image(moving_impaths, output_spacing=None, crop_depth=False, rescaling=True)
print(f"Original spacings: fixed image = {fixed_image.GetSpacing()}, moving image = {moving_image.GetSpacing()}")
print(f"Original sizes: fixed image = {fixed_image.GetSize()}, moving image = {moving_image.GetSize()}")